# Recreating the code

# Environmment related code

In [107]:
import numpy as np
import random


class DisasterZoneEnv:
    """
    A simplified 2D grid environment for a drone exploring a disaster zone.

    Legend:
        0 -> Empty cell
        1 -> Obstacle
        2 -> Survivor
        3 -> Recharging Point
        D -> Drone (display only during render)
    """

    def __init__(self, width=8, height=8, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=20, dynamic=False):
        """
        Initialize the environment with configurable dimensions and grid contents.
        """
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic
        self.energy = initial_energy
        self.dynamic_changes = 0  # Counter for dynamic changes
        self.reset()

    def reset(self):
        """
        Reset the environment to its initial state.
        """
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Place obstacles randomly
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1

        # Place survivors randomly
        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2

        # Place recharging points randomly
        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3

        # Set drone's starting position
        self.drone_x, self.drone_y = self._get_random_empty_cell()
        self.energy = self.initial_energy

    def _get_random_empty_cell(self):
        """
        Find a random empty cell in the grid.
        """
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:  # Check for empty cell
                return x, y

    def apply_dynamic_changes(self, step_count):
        """
        Apply dynamic changes to the environment.
        """
        if self.dynamic:
            # Add a new obstacle every 5 steps
            if step_count % 5 == 0:
                x, y = self._get_random_empty_cell()
                self.grid[x, y] = 1
                self.dynamic_changes += 1

            # Move survivors every 3 steps
            if step_count % 3 == 0:
                survivor_positions = [(x, y) for x in range(self.height)
                                      for y in range(self.width) if self.grid[x, y] == 2]
                for x, y in survivor_positions:
                    self.grid[x, y] = 0  # Remove survivor
                    new_x, new_y = self._get_random_empty_cell()
                    self.grid[new_x, new_y] = 2

    def render(self):
        """
        Render the grid for visualization.
        """
        grid_copy = self.grid.astype(str)
        grid_copy[grid_copy == '0'] = '.'
        grid_copy[grid_copy == '1'] = '#'
        grid_copy[grid_copy == '2'] = 'S'
        grid_copy[grid_copy == '3'] = 'R'
        grid_copy[self.drone_x, self.drone_y] = 'D'
        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")


# Agent related code

In [108]:
import heapq


class DijkstraAgent:
    """
    An agent that uses Dijkstra's algorithm to navigate the DisasterZoneEnv.
    """

    def __init__(self, env):
        self.env = env
        self.steps_taken = 0
        self.survivors_rescued = 0
        self.resources_collected = 0

    def dijkstra(self, grid, start, target):
        """
        Implements Dijkstra's algorithm to find the shortest path in a 2D grid.
        """
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        pq = [(0, start)]  # Priority queue (distance, position)
        distances = {start: 0}
        previous = {start: None}

        while pq:
            current_distance, current_position = heapq.heappop(pq)
            if current_position == target:
                path = []
                while current_position:
                    path.append(current_position)
                    current_position = previous[current_position]
                path.reverse()
                return path, current_distance

            for dx, dy in directions:
                neighbor = (current_position[0] + dx, current_position[1] + dy)
                if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                    if grid[neighbor[0], neighbor[1]] != 1:  # Ignore obstacles
                        new_distance = current_distance + 1
                        if neighbor not in distances or new_distance < distances[neighbor]:
                            distances[neighbor] = new_distance
                            previous[neighbor] = current_position
                            heapq.heappush(pq, (new_distance, neighbor))
        return [], float('inf')  # No valid path

    def find_closest_target(self, target_type):
        """
        Finds the closest target (survivor or resource).
        """
        start = (self.env.drone_x, self.env.drone_y)
        targets = [(x, y) for x in range(self.env.height) for y in range(self.env.width) if self.env.grid[x, y] == target_type]

        if not targets:
            return None, None, float('inf')

        shortest_path, closest_target, shortest_distance = None, None, float('inf')
        for target in targets:
            path, distance = self.dijkstra(self.env.grid, start, target)
            if distance < shortest_distance:
                shortest_path, closest_target, shortest_distance = path, target, distance

        return shortest_path, closest_target, shortest_distance

    def execute(self):
        """
        Executes the agent's logic in the environment.
        """
        step_count = 0
        while self.env.energy > 0:
            path, target_position, _ = self.find_closest_target(2)  # Look for survivors first
            if not path:
                path, target_position, _ = self.find_closest_target(3)  # Then look for resources

            if not path:
                print("No reachable targets. Stopping.")
                break

            for step in path[1:]:
                self.env.drone_x, self.env.drone_y = step
                self.steps_taken += 1
                step_count += 1

                if self.env.dynamic:
                    self.env.apply_dynamic_changes(step_count)

                self.env.energy -= 1
                if self.env.energy <= 0:
                    break

            if target_position:
                if self.env.grid[target_position[0], target_position[1]] == 2:
                    self.survivors_rescued += 1
                elif self.env.grid[target_position[0], target_position[1]] == 3:
                    self.resources_collected += 1
                self.env.grid[target_position[0], target_position[1]] = 0


# Tester related code

In [109]:
import time
import pandas as pd


class Tester:
    """
    Handles testing and metric collection for the agent and environment.
    """

    def __init__(self, num_tests, env_params, agent_class):
        self.num_tests = num_tests
        self.env_params = env_params
        self.agent_class = agent_class
        self.agent_name = agent_class.__name__  # Capture the name of the agent class
        self.results = []

    def run_test(self, dynamic):
        """
        Runs a single test for either static or dynamic environment.
        """
        env = DisasterZoneEnv(**self.env_params, dynamic=dynamic)
        agent = self.agent_class(env)
        start_time = time.time()
        agent.execute()
        computation_time = time.time() - start_time

        # Collect results, including initial environment parameters
        self.results.append({
            "Agent Name": self.agent_name,
            "Environment": "Dynamic" if dynamic else "Static",
            "Width": self.env_params["width"],
            "Height": self.env_params["height"],
            "Num Obstacles": self.env_params["num_obstacles"],
            "Num Survivors": self.env_params["num_survivors"],
            "Num Resources": self.env_params["num_resources"],
            "Initial Energy": self.env_params["initial_energy"],
            "Steps Taken": agent.steps_taken,
            "Survivors Rescued": agent.survivors_rescued,
            "Resources Collected": agent.resources_collected,
            "Energy Used": self.env_params["initial_energy"] - env.energy,
            "Dynamic Changes": env.dynamic_changes if dynamic else 0,
            "Computation Time (s)": computation_time
        })

    def run_all_tests(self):
        """
        Runs the specified number of tests for both static and dynamic environments.
        """
        for _ in range(self.num_tests):
            self.run_test(dynamic=False)  # Static environment test
            self.run_test(dynamic=True)   # Dynamic environment test

    def save_results(self, filename="results.csv"):
        """
        Saves the results to a CSV file.
        """
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")


# Running the code

In [110]:
if __name__ == "__main__":
    # Environment configuration
    env_params = {
        "width": 8,
        "height": 8,
        "num_obstacles": 5,
        "num_survivors": 3,
        "num_resources": 2,
        "initial_energy": 20
    }
    num_tests = 3  # Number of test iterations

    # Create a tester instance and test the DijkstraAgent
    tester = Tester(num_tests=num_tests, env_params=env_params, agent_class=DijkstraAgent)
    tester.run_all_tests()
    tester.save_results("test_metrics_with_env_params.csv")


No reachable targets. Stopping.
Results saved to test_metrics_with_env_params.csv
